In [255]:
# imports
from copy import copy, deepcopy
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split 
from sklearn import metrics
from sklearn import linear_model
import pandas as pd 
import numpy as np
from sklearn import metrics  
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from scipy.stats import randint as sp_randint
from sklearn.preprocessing import LabelEncoder
import random
import string

%matplotlib inline

In [256]:
# Load the Wine Dataset
df_wine = pd.read_csv("df_wine_country_typos.csv", encoding = 'utf8', index_col=0)
df_wine = df_wine.reset_index()
# check data
df_wine.head(10)

,level_0,index,country,province,region,price,variety,points,year_of_wine
0,0,0,UvA,Oregon,Willamette Valley,14.0,Pinot Gris,87,2013
1,1,1,US,Michigan,Lake Michigan Shore,13.0,Riesling,87,2013
2,2,2,USf,Oregon,Willamette Valley,65.0,Pinot Noir,87,2012
3,3,3,Spain,Northern Spain,Navarra,15.0,Tempranillo-Merlot,87,2011
4,4,4,Italy,Sicily & Sardinia,Vittoria,16.0,Frappato,87,2013
5,5,5,France,Alsace,Alsace,24.0,Gewürztraminer,87,2012
6,6,6,France,Alsace,Alsace,27.0,Pinot Gris,87,2012
7,7,7,US,California,Napa,19.0,Cabernet Sauvignon,87,2011
8,8,8,France,Alsace,Alsace,30.0,Gewürztraminer,87,2012
9,9,9,USZ,California,Sonoma,34.0,Cabernet Sauvignon,87,2012


In [257]:
# check distinct countries
countries = pd.unique(df_wine['country'])
print(countries)

['UvA' 'US' 'USf' ... 'UJVp' 'UogS' 'USrE']


In [258]:
# todo: check frequencies of country
frequencies = df_wine['country'].value_counts()
frequencies.sort_values
print(frequencies)

US           26558
France       16319
Italy        16069
Spain         6059
Argentina     3664
Australia     2223
Canada         253
USS             87
UcS             62
USD             60
USC             60
UzS             58
UiS             56
USq             55
USb             55
UyS             55
UOS             54
UDS             53
UrS             53
USo             53
UST             53
USF             53
USx             52
USA             52
UGS             52
UBS             52
USe             52
USN             52
USO             51
USm             51
             ...  
UJuA             1
UwhS             1
UTyB             1
UVkS             1
TES              1
UNdq             1
Use              1
SOSV             1
mSx              1
QSB              1
UPSS             1
UgeR             1
UyLS             1
zqS              1
Urdq             1
ULSC             1
MqPS             1
UYYp             1
UgZl             1
wRuS             1
UqpH             1
FlQS        

In [259]:
name = frequencies.index[0]
value = frequencies[0]
print("Most frequent: {name} ({value})".format(name=name, value=value))

Most frequent: US (26558)


# TYPOS DETECTION

In [260]:
def clean_value(value, original, percent):
    typos_chars = list(value)
    items = list(original)
    
    check_list = dict()
    for item in items:
        check_list[item] = False
    
    # check if value contains all chars of original
    for typo in typos_chars:
        if typo in original:
            if check_list[typo]:
                continue
            check_list[typo] = True
    
    # how many characters from the original were found, 
    # correctnes 100 means, all characters in the original were found
    correctness = int((percent * len(original)) / 100.0)
    corrected = len([x for x in check_list.values() if x])

    if corrected < correctness:
#         print("[ERROR] Couldnt clean: {}.".format(value))
        return value

    return original

In [261]:
def detect_typos(df, column_name, column_value, expected_distinct_values, correctness):
    i = 0
    b = 0
    print("[OK] Clean correctness: {}%".format(correctness))
    for index, row in df.iterrows():
        value = row[column_name]
        if value in expected_distinct_values:
            continue
        
        new_value = clean_value(value, column_value, correctness)
        
        if value != new_value:
            df.at[index, column_name] = new_value
#             print("[OK] Cleaning {} results in {}".format(value, new_value))
            i += 1
        else:
            # discard the row that couldnt be cleaned
            df.at[index, column_name] = np.NaN
            b += 1
    
    print("[OK] Typos detected: {}".format(b+i))
    print("[OK] Cleaned: {} rows".format(i))
    print("[WARN] Couldn't clean, removing: {} rows".format(b)) 
    return df

In [262]:
# Corrects detected typos and removes the typos that cannot be corrected
expected_distinct_values = ['US', 'Spain', 'Italy', 'France', 'Argentina', 'Australia', 'Canada']
correctness = 100

df_wine = detect_typos(df_wine, "country", "US", expected_distinct_values, correctness)

[OK] Clean correctness: 100%
[OK] Typos detected: 26558
[OK] Cleaned: 11798 rows
[WARN] Couldn't clean, removing: 14760 rows


In [263]:
print(df_wine.shape)

(97703, 9)


In [264]:
df_wine = df_wine.dropna()

In [265]:
print(df_wine.shape)

(82943, 9)


In [266]:
# check results
frequencies = df_wine['country'].value_counts()
frequencies.sort_values

print(frequencies)

US           38356
France       16319
Italy        16069
Spain         6059
Argentina     3664
Australia     2223
Canada         253
Name: country, dtype: int64


In [267]:
# Auto encodes any dataframe column of type category or object.
def dummyEncode(df):
        columnsToEncode = list(df.select_dtypes(include=['category','object']))
        le = LabelEncoder()
        for feature in columnsToEncode:
            try:
                df[feature] = le.fit_transform(df[feature])
            except:
                print('Error encoding '+feature)
        return df

In [268]:
# one hot encoding
df_dummy = dummyEncode(df_wine[['country','province','region','variety']])
df_dummy.columns = ['country_num', 'province_num','region_num','variety_num']
df_wine = pd.concat([df_wine, df_dummy], axis=1)

/Users/roxana/.virtualenvs/dojo3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [269]:
df_wine.head(4)
# us country num = 6

,level_0,index,country,province,region,price,variety,points,year_of_wine,country_num,province_num,region_num,variety_num
1,1,1,US,Michigan,Lake Michigan Shore,13.0,Riesling,87,2013,6,27,471,336
2,2,2,US,Oregon,Willamette Valley,65.0,Pinot Noir,87,2012,6,40,997,311
3,3,3,Spain,Northern Spain,Navarra,15.0,Tempranillo-Merlot,87,2011,5,36,627,416
4,4,4,Italy,Sicily & Sardinia,Vittoria,16.0,Frappato,87,2013,4,47,987,126


In [270]:
# Variety
df_variety = df_wine.variety.str.get_dummies()
df_variety.columns = ['v_' + col for col in df_variety.columns] # naming the columns
df_wine = pd.concat([df_wine, df_variety], axis=1) # combine main dataframe with country matrix dataframe
df_wine = df_wine.drop('variety', axis=1) # drop the country attribute since different attributes per country value
df_wine = df_wine.drop('variety_num', axis=1) # drop the country attribute since different attributes per country value

In [271]:
# Country
df_country = df_wine.country.str.get_dummies() # get the pivot of the country attribte
df_country.columns = ['country_' + col for col in df_country.columns] # naming the columns
df_wine = pd.concat([df_wine, df_country], axis=1) # combine main dataframe with country matrix dataframe
df_wine = df_wine.drop('country', axis=1) # drop the country attribute since different attributes per country value
df_wine = df_wine.drop('country_num', axis=1) # drop the country attribute since different attributes per country value

In [272]:
# Province
df_province = df_wine.province.str.get_dummies() # get the pivot of the country attribte
df_province.columns = ['province_' + col for col in df_province.columns] # naming the columns
df_wine = pd.concat([df_wine, df_province], axis=1) # combine main dataframe with country matrix dataframe
df_wine = df_wine.drop('province', axis=1) # drop the country attribute since different attributes per country value
df_wine = df_wine.drop('province_num', axis=1) # drop the country attribute since different attributes per country value

In [273]:
# Region
df_wine = df_wine.drop('region', axis=1) # drop the country attribute since different attributes per country value
df_wine = df_wine.drop('region_num', axis=1) # drop the country attribute since different attributes per country value

In [274]:
df_wine.head(5)

,level_0,index,price,points,year_of_wine,v_Abouriou,v_Aglianico,v_Airen,v_Albana,v_Albanello,...,province_Spanish Islands,province_Tasmania,province_Texas,province_Tuscany,province_Veneto,province_Vermont,province_Victoria,province_Virginia,province_Washington,province_Western Australia
1,1,1,13.0,87,2013,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,2,65.0,87,2012,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,3,15.0,87,2011,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,4,16.0,87,2013,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,5,5,24.0,87,2012,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [275]:
# check new headers
headers = list(df_wine)

for header in headers:
    if "country_" in header:
        print(header)

country_Argentina
country_Australia
country_Canada
country_France
country_Italy
country_Spain
country_US


In [276]:
# check only numbers in data
df_wine.head(5)

,level_0,index,price,points,year_of_wine,v_Abouriou,v_Aglianico,v_Airen,v_Albana,v_Albanello,...,province_Spanish Islands,province_Tasmania,province_Texas,province_Tuscany,province_Veneto,province_Vermont,province_Victoria,province_Virginia,province_Washington,province_Western Australia
1,1,1,13.0,87,2013,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,2,65.0,87,2012,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,3,15.0,87,2011,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,4,16.0,87,2013,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,5,5,24.0,87,2012,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [277]:
# split data
df_X = df_wine.drop('points', axis=1)
df_Y = df_wine[['points']]
X_train, X_test, y_train, y_test = train_test_split(df_X, df_Y, test_size=0.3, random_state=0) 

In [278]:
def get_hyperparameters(X_train, y_train):
    regressor3 = DecisionTreeRegressor()
    param_dist = {'max_depth': sp_randint(2,16),
                  'min_samples_split': sp_randint(2,16)}

    n_iter_search = 20
    clfrs = RandomizedSearchCV(regressor3, 
                               param_distributions=param_dist,
                               scoring='neg_mean_squared_error',
                               cv=5 , n_jobs=1, verbose=1,
                               n_iter=n_iter_search)
    clfrs.fit(X_train, y_train)
    params = clfrs.best_params_
    return params["max_depth"], params["min_samples_split"]

In [ ]:
# get hyperparameters
max_depth, min_samples_split = get_hyperparameters(X_train, y_train)
print("max_depth: {}, min_samples_split: {}".format(max_depth, min_samples_split)) 

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


# RESULT TYPOS CLEAN

In [ ]:
regressor = DecisionTreeRegressor(max_depth=max_depth, min_samples_split=min_samples_split)  

# Train the model using the training sets
regressor.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = regressor.predict(X_test)

# The evaluation metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred))) 